In [5]:
import torchfile
import numpy as np
import time, sys
import os

from keras.regularizers import l2
from __future__ import print_function
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np
import resnet
import tensorflow as tf

# Load Data

In [6]:
train_data = torchfile.load('/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/trainobjcat.t7')
test_data = torchfile.load('/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/valobjcat.t7')

# Test and Train Data

In [7]:
X_train = train_data[b'x'][0]
y_train = train_data[b'y']
X_test = test_data[b'x'][0]
y_test = test_data[b'y']
X_train=np.swapaxes(np.swapaxes(X_train, 1, 2), 2, 3)
X_test=np.swapaxes(np.swapaxes(X_test, 1, 2), 2, 3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6415, 128, 128, 3), (6415,), (1604, 128, 128, 3), (1604,))

In [8]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet18_robot.csv')

batch_size = 32
nb_classes = 282
nb_epoch = 200
data_augmentation = False

# input image dimensions
img_rows, img_cols = 128, 128
# The CIFAR10 images are RGB.
img_channels = 3

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
#mean_image = np.mean(X_train, axis=0)
#X_train -= mean_image
#X_test -= mean_image
#X_train /= 128.
#X_test /= 128.

model = resnet.ResnetBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

# Model Architecture

# Fit Model

In [ ]:
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
with tf.device("/gpu:0"):
    model.fit(X_train, Y_train, validation_split=.1, epochs=nb_epoch, batch_size=32)

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


Train on 5773 samples, validate on 642 samples
Epoch 1/200
5773/5773 [==============================] - 1172s - loss: 4.8467 - acc: 0.1502 - val_loss: 6.4103 - val_acc: 0.0483
Epoch 2/200
5773/5773 [==============================] - 1164s - loss: 2.9723 - acc: 0.4152 - val_loss: 7.4376 - val_acc: 0.0748
Epoch 3/200
5773/5773 [==============================] - 1165s - loss: 2.2011 - acc: 0.5782 - val_loss: 13.3965 - val_acc: 0.0405
Epoch 4/200
5773/5773 [==============================] - 1133s - loss: 1.7656 - acc: 0.6849 - val_loss: 11.8058 - val_acc: 0.0654
Epoch 5/200
5773/5773 [==============================] - 1019s - loss: 1.5020 - acc: 0.7518 - val_loss: 8.4733 - val_acc: 0.1199
Epoch 6/200
5773/5773 [==============================] - 1026s - loss: 1.2959 - acc: 0.7996 - val_loss: 13.1233 - val_acc: 0.0639
Epoch 7/200
5773/5773 [==============================] - 972s - loss: 1.2112 - acc: 0.8164 - val_loss: 9.8304 - val_acc: 0.1262
Epoch 8/200
5773/5773 [=========================

In [ ]:
DATA_DIR = os.getcwd().rsplit('/', 1)[0] + '/data'
checkpoint_dir = os.getcwd().rsplit('/', 1)[0] + '/checkpoint'